## Import All Packages

In [2]:
import numpy as np
import re
import string
import os
import hashlib
import sortedcontainers
from sortedcontainers import SortedDict
import pickle
dictionary = SortedDict()

C:\Users\untro\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\untro\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\Users\untro\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
%load_ext line_profiler

In [4]:
%load_ext memory_profiler

## Hash Function

In [15]:
def string2numeric_hash(text):
    import hashlib
    return int(hashlib.sha256(text.encode("utf-8")).hexdigest()[:16], 16)

## Create Boolean Index

In [70]:
def read_tokens_file(file_name):
    with open(file_name,'r',encoding = "utf-8") as stream:
        stream_ended = False
        while True:
            doc_id = int(stream.readline().split(":")[1].strip())
            length = int(stream.readline().strip())
            for idx in range(length):
                string = stream.readline()
                hash_code = string2numeric_hash(string.strip())
                dictionary.setdefault(key = hash_code,default=[])
                dictionary[hash_code].append(doc_id)
            if doc_id == 39999:
                break
    for key in dictionary.keys():
        dictionary[key] = sorted(list(set(dictionary[key])))

def read_token_file_mapping(file_name,file_pattern):
    with open(file_name,'r',encoding = 'utf-8') as stream:
        map_dict = SortedDict()
        stream_ended = False
        while True:
            doc_id = int(stream.readline().split(":")[1].strip())
            length = int(stream.readline().strip())
            for idx in range(length):
                string = stream.readline()
                map_dict.setdefault(key = string,default = [])
                map_dict[string].append(doc_id)
            if doc_id == 39999:
                break
        for key in dictionary.keys():
            dictionary[key] = sorted(list(set(dictionary[key])))
        with open(file_pattern + "_dict.pickle",'wb') as dict_stream:
            strings = map_dict.keys()
            pickle.dump(list(strings),dict_stream)
        with open(file_pattern + "_coord.pickle",'wb') as coord_dict:
            coordinates = map_dict.values()
            pickle.dump(list(coordinates),coord_dict)
                
        

In [71]:
%time read_tokens_file("tokens_upd.txt")

Wall time: 4min 36s


In [59]:
%time read_token_file_mapping("tokens_upd.txt","table")

['a_\n', 'a_dict\n', 'a_gaidar\n', 'a_i\n', 'a_j\n', 'a_n\n', 'a_port\n', 'a_prototyp_aa\n', 'aa\n', 'aaa\n']
Wall time: 2min 3s


## Save boolean index(tree variant)

In [60]:
import pickle

In [21]:
with open("table.pickle",'wb') as storage:
    pickle.dump(dictionary,storage)

In [62]:
%memit dictionary

peak memory: 733.65 MiB, increment: 0.05 MiB


In [7]:
with open("table.pickle",'rb') as storage:
    dictionary = pickle.load(storage)

In [6]:
keys = dictionary.keys()[:20]
values = dictionary.values()[:20]

In [74]:
for key,value in zip(keys,values):
    print("hash_string: {0} - doc_id:{1}".format(key,value))

hash_string: 4930 - doc_id:[854, 17744, 30267, 37488]
hash_string: 6476 - doc_id:[19296]
hash_string: 11145 - doc_id:[3995, 11976, 19726, 26834, 33496, 36333]
hash_string: 12460 - doc_id:[31225]
hash_string: 12746 - doc_id:[1136, 1261, 1286, 1721, 1933, 2686, 4500, 5661, 5670, 6166, 6197, 8607, 9297, 9486, 9763, 9765, 9980, 10026, 10041, 11200, 11519, 11726, 13375, 14819, 15734, 16410, 16721, 17565, 17747, 17860, 18237, 18897, 19144, 19297, 20391, 20976, 24794, 25216, 25801, 28147, 28925, 30268, 31964, 33136, 33586, 34218, 34240, 35716, 36212, 36665, 38367, 38825, 39105, 39799]
hash_string: 13910 - doc_id:[5562, 10702, 13329, 23194, 27839, 31853, 33535, 34284, 34902]
hash_string: 16283 - doc_id:[7013, 9785, 21720, 36325, 36515, 37726, 38709, 39563, 39593, 39937]
hash_string: 23417 - doc_id:[10930]
hash_string: 25257 - doc_id:[5521]
hash_string: 26309 - doc_id:[39, 561, 763, 765, 766, 777, 815, 1026, 1056, 1106, 1203, 1210, 1211, 1213, 1310, 1346, 2170, 2419, 2459, 2493, 2583, 4008, 453

## Test Case

In [11]:
text_token = os.path.join("test","wiki1201.txt")
text_token1 = os.path.join("test","wiki10201.txt")
text_token2 = os.path.join("test","wiki0.txt")

In [12]:
tokens = []
with open(text_token,'r',encoding=  'utf-8') as stream:
    for line in stream:
        tokens.append(line)       
with open(text_token1,'r',encoding=  'utf-8') as stream:
    for line in stream:
        tokens.append(line)
with open(text_token2,'r',encoding=  'utf-8') as stream:
    for line in stream:
        tokens.append(line)

In [13]:
tokens = list(map(lambda x:x.strip(),tokens))

In [20]:
for token in tokens:
    hash_string = string2numeric_hash(token)
    print("________________________________________________________________________________________________________________")
    print("in 1201: " + str(1201 in dictionary[hash_string]))
    print("in 10201: " + str(10201 in dictionary[hash_string]))
    print("in 0: " + str(0 in dictionary[hash_string]))
    if not ((1201 in dictionary[hash_string]) or (10201 in dictionary[hash_string]) or (0 in dictionary[hash_string])):
        raise RuntimeError("Эта юниграмма должна быть в этом файле")
print("Success")


________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
_______________________________________________________________________

in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
_________________________________________________

in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
______________________________

in 1201: True
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
______________________________

________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: False
______________________________________________________________________

...

________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
__________________________________________________________________________

________________________________________________________________________________________________________________
in 1201: True
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: True
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
___________________________________________________________________________

in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
_____________________________________________________________

in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
___________________________________________________________

in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: False
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
________________________________________________________________________________________________________________
in 1201: True
in 10201: False
in 0: True
_____________________________________________